# Вместо преамбулы

Давайте вместе представим, что нам срочно нужно узнать город по координатам. Причины могут быть любыми: удивить обидчика, продать пользователю что-нибудь ненужное, выбрать место для следующего путешествия или просто побаловаться. Для этого нам нужно решить несколько задач:

1. Так как мы находимся на шаре (точнее геоиде), то нужно отобразить координаты на плоскость или как-то учитывать это при расчетах. Не все помнят, что в зависимости от широты в одном градусе разное число километров (за нас это будет решать spatial движок postgis, спасибо ему большое).
2. Посчитать попала ли точка в невыпуклый многоугольник. Нетривиальная, кстати, задача. Интересующиеся могут пройти на [вики](https://en.wikipedia.org/wiki/Point_in_polygon).
3. Понять к чему этот многоугольник относится.

В этой тетрадке я вкратце пройдусь по решению этой задачи, объясняя все настолько доступно, насколько могу. *Опытные маркшейдеры, пожалуйста, не смейтесь, а лучше внесите правки в дилетантский текст.*

## Подготовка

### Немного терминов:
* Атрибутивная информация - собственно, качественные и количественные характеристики объекта. Обычная таблица, короче. Через атрибуты мы будем определять что это за полигон.
* Пространственная информация - данные о геометрии объекта. Через геометрию мы будем определять куда же мы таки попали.
* CRS (Coorsinate Reference System) - система координат для пространственных данных. Систем координат [превеликое множество](), но самых распространненых в вебе две: [WGS84](https://en.wikipedia.org/wiki/World_Geodetic_System) и [Web Mercator](https://en.wikipedia.org/wiki/Web_Mercator). Для простоты мы будем пользоваться географическими координатами WGS84, желающие могут потренироваться в других системах. Интересующимся на [EPSG](http://epsg.io/).


### Подготовка БД
Для всех манипуляций я буду использовать Postgres + PostGIS для хранения атрибутивной и пространственной информации, а питон и sql для запросов.

#### PostGIS.
Самым простым способом получить себе настроенный PostGIS - запустить уже готовый докер контейнер. Так и поступим. Я взял [postgis](https://hub.docker.com/r/mdillon/postgis/).

`docker run --name some-postgis -e POSTGRES_PASSWORD=mysecretpassword -d mdillon/postgis`

#### Данные.
Для начала в наш контейнер нужно положить файлик с данными. Данные возьмем с [http://osm2shp.ru/](http://osm2shp.ru/), конкретно нас интересует *boundary_ply* (датасет довольно большой, но тут все границы объектов в Евразии, извините). Я скачивал в tmp просто через `wget`. После этого распаковываем архив и заглянем внутрь.
![Архив](https://i.imgur.com/LNqbmbq.png)

Нас интересуют три файла:

1. `boundary_ply.shp` - тут лежит геометрия.

2. `boundary_ply.dgf` - тут лежит атрибутивная информация.

3. `boundary_ply.prj` - тут лежит информация о системе координат. Если глянуть на этот файл через `cat`, то мы увидим описание системы в [WKT](https://en.wikipedia.org/wiki/Well-known_text) - `GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]]`. Это обычный WGS84, нам подходит.

Создаем базу ([документация](/psto.png)):

1. `createdb shape`

2. `psql -d shape -c "CREATE EXTENSION postgis;"`

3. `psql -d shape -c "CREATE EXTENSION postgis_topology;"` 

4. Загружаем данные: `shp2pgsql -I boundary_ply.shp public.boundaries | psql -d shape`

### Подготовка Python

Чтобы работать с postgis нужно помимо обычного psql драйвера поставить что-нибудь для postgis. В питоне для этого можно взять одноименную библиотеку. Я буду использовать его вместе с psycopg2. 

`pip install postgis`

`pip install psycopg2`

Все, теперь мы можем баловаться с нашими данными сколько влезет.


## Модель данных

Итак, давайте взглянем как выглядит наша таблица:

```
Table "public.boundaries"
   Column   |          Type          |                        Modifiers
------------+------------------------+----------------------------------------------------------
 gid        | integer                | not null default nextval('boundaries_gid_seq'::regclass)
 relationid | character varying(11)  |
 wayid      | character varying(11)  |
 boundary   | character varying(25)  |
 admin_lvl  | character varying(20)  |
 border_tp  | character varying(20)  |
 is_in      | character varying(119) |
 country    | character varying(26)  |
 state      | character varying(27)  |
 maritime   | character varying(20)  |
 name       | character varying(254) |
 fips_code  | character varying(20)  |
 state_fips | character varying(20)  |
 place      | character varying(20)  |
 population | character varying(20)  |
 postal_cd  | character varying(54)  |
 geom       | geometry(MultiPolygon) |
Indexes:
    "boundaries_pkey" PRIMARY KEY, btree (gid)
    "boundaries_geom_idx" gist (geom)
```

Здесь нас всерьез интересуют четыре поля:

1. geom - собственно, геометрия.

2. name - имя полигона.

3. boundary - тип границы. 

4. admin_lvl - тип объекта. NB! Для каждой страны свое, поэтому лучше посмотреть в вики.

Для городов в OSM используется тип `administrative` и `admin_lvl` 4 и 8. (3 города федерального значения + "обычные" города) [OSM Wiki](http://wiki.openstreetmap.org/wiki/Tag:boundary%3Dadministrative).

Протестируем предположение, ткнув в центр Петербурга (из консоли psql):

```
select
  name
from public.boundaries
where
  boundary = 'administrative'
  and admin_lvl = '4' or admin_lvl = '8'
  and ST_Contains(geom, st_geomfromtext('POINT(30.1855915 59.9099508)'));
```



```
name
-------------------------------
Санкт-Петербург
(1 row)

```
  
[Подозрительно похоже на правду](https://www.google.ru/maps/place/Saint+Petersburg/@59.9099508,30.1855915,11.5z/data=!4m5!3m4!1s0x4696378cc74a6f9d:0xfa47ca5a4725c9e4!8m2!3d60.0762383!4d30.1213829?hl=en). Кстати, Девяткино - не Питер.

Да, не стоит удивляться, что мы переставили широту и долготу. Для postgis это x и y.

Здесь мы использовали функции ST_Contains и ST_GeomFromText (а дальше будем пользовать только ST_Contains). [ST_Contains(A, B)](http://postgis.net/docs/manual-1.4/ST_Contains.html), как следует из названия, проверяет содержатся ли в полигоне A все точки из полигона B. Нам подходит.

Теперь давайте то же самое сделаем из питона.

In [24]:
import psycopg2
from postgis.psycopg import register
from postgis import Point

CONN_STR = "host='192.168.99.100' port='32768' dbname='shape' user='postgres' password='postgres'"
QUERY = "select name from public.boundaries where boundary = 'administrative' and (admin_lvl = '4' or admin_lvl = '8') and ST_Contains(geom, %s)"

def hittest(lat, lon):
    with psycopg2.connect(CONN_STR) as conn:
        register(conn)
        
        cursor = conn.cursor()
        
        cursor.execute(QUERY, [Point(x = lon, y = lat)])
        
        for e in cursor.fetchone():
            print(e)

hittest(59.90995, 30.18559)
hittest(55.6209819, 37.6372982)
hittest(51.790602, 143.1082624)

Санкт-Петербург
Москва
Сахалинская область


Когда-нибудь потом я покажу как наш датасет опубликовать в GeoServer и сделать симпатичный интерфейс на leaflet.js. Поверьте, это так же просто.